In [1]:
import pymongo
import os
import json
import asyncio
import aiohttp

In [2]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
uri = os.environ.get('MONGO_URI')

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [3]:
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

In [4]:
client = pymongo.MongoClient(uri)
db = client[os.environ.get('DATABASE')]
collection = db[os.environ.get('COLLECTION')]

In [5]:
json.dumps({"query": "My name is neel"})


'{"query": "My name is neel"}'

In [13]:
import aiohttp
import json

async def generate_embedding() -> list[float]:
  url = "http://20.197.8.42:5000/preprocess"
  headers = {"Content-Type": "application/json"}
  async with aiohttp.ClientSession() as session:
    async with session.post(url, headers=headers, json=json.loads(json.dumps({"query": "Best dosa in delhi"}))) as response:
      json_data = await response.json()
      print(json_data)
      return json_data['embeddings']

query = "flight"
embedding_query = await generate_embedding()


{'embeddings': [0.2927424907684326, -0.20540016889572144, -0.44871634244918823, -1.0415441989898682, -0.6859978437423706, 2.062631607055664, -0.024532930925488472, -0.12861913442611694, 0.6448041200637817, -0.44494083523750305, -1.7338449954986572, 0.6615448594093323, -0.5958438515663147, 0.8156431913375854, 0.39232391119003296, -0.39532750844955444, -1.0075445175170898, 0.22152191400527954, -0.6336354613304138, -0.7509833574295044, 0.40518563985824585, -2.2746429443359375, -0.8995946645736694, 0.14764142036437988, -0.2301032692193985, -0.4910641312599182, 0.4960829019546509, 0.35346317291259766, 0.46312853693962097, 0.18258824944496155, 1.8246455192565918, -0.3365567922592163, -1.062028408050537, -1.6835074424743652, 1.6165270805358887, 0.4054352343082428, 1.0660918951034546, 1.2308154106140137, -0.8597005605697632, -0.05013060197234154, 0.2961746156215668, -0.3115258812904358, -0.7209892272949219, -0.07680751383304596, 2.6381776332855225, -1.0131369829177856, -1.6335123777389526, -0.

In [14]:
len(embedding_query)

768

In [15]:
results = collection.aggregate([
  {"$vectorSearch": {
    "queryVector": embedding_query,
    "path": "embedding",
    "numCandidates": 100,
    "limit": 4,
    "index": "vector_index",
      }}
])

In [16]:
for document in results:
    # print(f'Movie Name: {document["title"]},\nMovie Plot: {document["plot"]}\n')
    print(document)

{'_id': ObjectId('660c16b94a0506a73605e6d2'), 'Restaurant Name': 'Flavors', 'Location': 'Chennai', 'Cuisine': 'Hyderabadi', 'Secondary Cuisine': 'Mughlai', 'Price Range': '₹500-₹1000', 'Dish': 'Dhokla', 'Price': 127, 'embedding': [0.2927424907684326, -0.20540016889572144, -0.44871634244918823, -1.0415441989898682, -0.6859978437423706, 2.062631607055664, -0.024532930925488472, -0.12861913442611694, 0.6448041200637817, -0.44494083523750305, -1.7338449954986572, 0.6615448594093323, -0.5958438515663147, 0.8156431913375854, 0.39232391119003296, -0.39532750844955444, -1.0075445175170898, 0.22152191400527954, -0.6336354613304138, -0.7509833574295044, 0.40518563985824585, -2.2746429443359375, -0.8995946645736694, 0.14764142036437988, -0.2301032692193985, -0.4910641312599182, 0.4960829019546509, 0.35346317291259766, 0.46312853693962097, 0.18258824944496155, 1.8246455192565918, -0.3365567922592163, -1.062028408050537, -1.6835074424743652, 1.6165270805358887, 0.4054352343082428, 1.066091895103454